In [2]:
!pip install -q torch_geometric
import torch
import numpy as np
from torch_geometric.datasets import Planetoid
from sklearn.metrics.pairwise import cosine_similarity
from torch_geometric.utils import dense_to_sparse
import networkx as nx
import os
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# Step 1: Load the Dataset (Generalized for Cora, Citeseer, and Pubmed)
def load_dataset(name):
    dataset = Planetoid(root=f'/tmp/{name}', name=name)
    data = dataset[0]
    return data

# Step 2: Create the First View (Citation Graph)
def create_first_view(data):
    # The first view is already provided by the dataset as the citation graph.
    first_view_edge_index = data.edge_index
    return first_view_edge_index

# Step 3: Create the Second View (Feature Similarity Graph)
def create_second_view(data, threshold=0.7):
    # Compute cosine similarity between node features
    features = data.x.numpy()
    similarity_matrix = cosine_similarity(features)
    
    # Apply a threshold to construct the graph
    num_nodes = similarity_matrix.shape[0]
    edge_index = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and similarity_matrix[i, j] > threshold:
                edge_index.append([i, j])
    
    second_view_edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return second_view_edge_index

# Step 4: Create the Third View (b-Matching Graph)
def create_third_view(data, k=10):
    # Using NetworkX for b-Matching
    G = nx.Graph()
    features = data.x.numpy()
    
    # Add nodes
    for i in range(features.shape[0]):
        G.add_node(i)
    
    # Add edges based on similarity (without threshold, just top-k similar)
    similarity_matrix = cosine_similarity(features)
    for i in range(similarity_matrix.shape[0]):
        similar_nodes = np.argsort(-similarity_matrix[i, :])[:k]
        for j in similar_nodes:
            if i != j:
                G.add_edge(i, j, weight=similarity_matrix[i, j])
    
    # Convert to PyTorch geometric format
    adj_matrix = nx.adjacency_matrix(G).todense()
    third_view_edge_index, _ = dense_to_sparse(torch.tensor(adj_matrix, dtype=torch.float))
    return third_view_edge_index

# Step 5: Preprocess Dataset (Generalized for Cora, Citeseer, and Pubmed)
def preprocess_dataset(name):
    data = load_dataset(name)
    
    # Generate the views
    first_view = create_first_view(data)
    second_view = create_second_view(data, threshold=0.7)
    third_view = create_third_view(data, k=10)
    
    # Store processed data in a dictionary
    processed_data = {
        'first_view': first_view,
        'second_view': second_view,
        'third_view': third_view,
        'features': data.x,
        'labels': data.y,
        'train_mask': data.train_mask,
        'val_mask': data.val_mask,
        'test_mask': data.test_mask
    }

    return processed_data

# Step 6: Execute Preprocessing and Store in a Dictionary for All Datasets
if __name__ == "__main__":
    all_datasets = {}
    for dataset_name in ['Cora', 'Citeseer', 'Pubmed']:
        print(f"Processing {dataset_name} dataset...")
        processed_data = preprocess_dataset(dataset_name)
        all_datasets[dataset_name] = processed_data

    print("All datasets have been processed and stored in the 'all_datasets' dictionary.")


Processing Cora dataset...


Processing...
Done!


Processing Citeseer dataset...


Processing...
Done!


Processing Pubmed dataset...


Processing...
Done!


All datasets have been processed and stored in the 'all_datasets' dictionary.


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch_geometric
from torch_geometric.utils import add_self_loops, degree

In [88]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.utils import scatter
import torch.nn.init as init
import torch.nn as nn

class MAGCN(torch.nn.Module):
    def __init__(self, num_features, num_classes, num_views, hidden_dim=16, dropout=0.5):
        super(MAGCN, self).__init__()
        self.num_views = num_views
        self.dropout = dropout

        self.gcn_unfold = torch.nn.ModuleList([
            GCNConv(num_features, hidden_dim) for _ in range(num_views)
        ])
        self.relu = nn.ReLU()
        self.dropout_layer = nn.Dropout(self.dropout)
        
        # MLP for attention mechanism with dropout
        self.attention_mlp = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, 6),
            torch.nn.ReLU(),
            nn.Dropout(self.dropout),  # Apply dropout
            torch.nn.Linear(6, 3),
            torch.nn.ReLU(),
            nn.Dropout(self.dropout),  # Apply dropout
            torch.nn.Linear(3, num_views)
        )
        
        # Final GCN layer after merging with dropout
        self.gcn_merge = GCNConv(hidden_dim, num_classes)
        self.final_dropout = nn.Dropout(self.dropout)  # Apply dropout after the final GCN layer

        # Initialize weights for Linear and GCNConv layers using Glorot (Xavier) initialization
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.zeros_(m.bias)
            elif isinstance(m, GCNConv):
                init.xavier_uniform_(m.lin.weight)
                if m.lin.bias is not None:
                    init.zeros_(m.lin.bias)

    def graph_gap(self, x, edge_index):
        """Graph Global Average Pooling operation"""
        # Add self-loops to ensure self-connections
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        # Get the degree of each node (including self-loops)
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype) + 1  # Degree + self-loop
        
        # Normalize node features by the degree (mean aggregation)
        deg_inv = deg.pow(-1).view(-1, 1)
        x = deg_inv * x
        
        # Sum the features from the neighbors
        out = scatter(x[col], row, dim=0, dim_size=x.size(0), reduce='add')
        
        return out

    def forward(self, data):
        # Step 1: Apply GCN to each view
        view_outputs = []
        for i in range(self.num_views):
            x = self.gcn_unfold[i](data.x, data.view_edge_index[i])
            x = self.relu(x)
            x = self.dropout_layer(x)
            view_outputs.append(x)

        # Step 2: Apply Graph GAP for each view
        gap_outputs = []
        for i in range(self.num_views):
            gap_output = self.graph_gap(view_outputs[i], data.view_edge_index[i])
            gap_outputs.append(gap_output)

        gap_outputs = torch.stack(gap_outputs, dim=0)  # Shape: (num_views, num_nodes, hidden_dim)

        # Step 3: Compute attention scores using MLP
        pooled_gap = torch.mean(gap_outputs, dim=0)  # Shape: (num_nodes, hidden_dim)

        attention_scores = self.attention_mlp(pooled_gap)  # Shape: (num_nodes, num_views)

        # Normalize attention scores across views for each node
        attention_scores = F.softmax(attention_scores, dim=1)  # Shape: (num_nodes, num_views)

        # Step 4: Apply attention to the view outputs
        attention_scores = attention_scores.unsqueeze(-1)  # Shape: (num_nodes, num_views, 1)
        weighted_output = torch.sum(attention_scores * gap_outputs.permute(1, 0, 2), dim=1)  # Shape: (num_nodes, hidden_dim)

        # Step 5: Merge the views with final GCN
        out = self.gcn_merge(weighted_output, data.view_edge_index[0])  # Use first view's edges for merging
        out = self.final_dropout(out)
        return F.log_softmax(out, dim=1)


In [89]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
import os

# Function to train the MAGCN model with early stopping
def train_model(model, data, optimizer, epochs=2000, early_stopping_patience=100, model_save_path="best_model.pth"):
    model.train()
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Train accuracy
        _, pred_train = out[data.train_mask].max(dim=1)
        correct_train = int((pred_train == data.y[data.train_mask]).sum())
        train_acc = correct_train / int(data.train_mask.sum())

        # Validation step
        model.eval()  # Switch to evaluation mode
        with torch.no_grad():
            val_out = model(data)
            val_loss = F.nll_loss(val_out[data.val_mask], data.y[data.val_mask])

            # Validation accuracy
            _, pred_val = val_out[data.val_mask].max(dim=1)
            correct_val = int((pred_val == data.y[data.val_mask]).sum())
            val_acc = correct_val / int(data.val_mask.sum())
        
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Acc: {train_acc:.4f}, Val_Loss: {val_loss.item():.4f}, Val_Acc: {val_acc:.4f}')

        # Early stopping and save the best model
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            best_model_state = model.state_dict()

            # Save the best model state to a file
            torch.save(best_model_state, model_save_path)
        else:
            patience_counter += 1
        
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered.")
            break

        model.train()  # Switch back to training mode
    
    # Load the best model after training
    model.load_state_dict(torch.load(model_save_path))
    print(f"Best model loaded from {model_save_path}")

# Function to test the MAGCN model
def test_model(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = int((pred[data.test_mask] == data.y[data.test_mask]).sum())
    acc = correct / int(data.test_mask.sum())
    print(f'Test Accuracy: {acc:.4f}')

# Modified data loading to read from dictionary and structure it for MAGCN
def load_processed_data_from_dict(dataset_dict):
    return Data(
        x=dataset_dict['features'],
        y=dataset_dict['labels'],
        train_mask=dataset_dict['train_mask'],
        val_mask=dataset_dict['val_mask'],
        test_mask=dataset_dict['test_mask'],
        view_edge_index=[dataset_dict['first_view'], dataset_dict['second_view'], dataset_dict['third_view']]
    )




In [91]:

if __name__ == "__main__":
    # Assume all_datasets dictionary is already populated as in the previous steps
    dataset_name = 'Cora'  # Change this to Citeseer or Pubmed as needed
    data = load_processed_data_from_dict(all_datasets[dataset_name])

    # Model configuration based on the paper's setup
    num_features = data.x.size(1)
    num_classes = int(data.y.max()) + 1
    hidden_dim = 16  # Set the hidden dimension to 16 as per the paper
    num_views = 3

    # Initialize the model with the specified architecture and dimensions
    model = MAGCN(num_features, num_classes, num_views, hidden_dim)
    
    # Use Adam optimizer with the specified learning rate and weight decay
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # Train and test the model
    train_model(model, data, optimizer, epochs=2000, early_stopping_patience=1000)
    test_model(model, data)

Epoch 1, Loss: 1.9446, Acc: 0.1714, Val_Loss: 1.9183, Val_Acc: 0.3000
Epoch 2, Loss: 1.9132, Acc: 0.2500, Val_Loss: 1.8755, Val_Acc: 0.5720
Epoch 3, Loss: 1.8242, Acc: 0.4214, Val_Loss: 1.8209, Val_Acc: 0.6720
Epoch 4, Loss: 1.7822, Acc: 0.4214, Val_Loss: 1.7661, Val_Acc: 0.6900
Epoch 5, Loss: 1.7421, Acc: 0.3643, Val_Loss: 1.7084, Val_Acc: 0.7240
Epoch 6, Loss: 1.6455, Acc: 0.4714, Val_Loss: 1.6558, Val_Acc: 0.7540
Epoch 7, Loss: 1.6250, Acc: 0.4286, Val_Loss: 1.6135, Val_Acc: 0.7660
Epoch 8, Loss: 1.5155, Acc: 0.4714, Val_Loss: 1.5650, Val_Acc: 0.7600
Epoch 9, Loss: 1.4300, Acc: 0.5071, Val_Loss: 1.5218, Val_Acc: 0.7360
Epoch 10, Loss: 1.3775, Acc: 0.5357, Val_Loss: 1.4697, Val_Acc: 0.7360
Epoch 11, Loss: 1.3929, Acc: 0.4857, Val_Loss: 1.4155, Val_Acc: 0.7520
Epoch 12, Loss: 1.2936, Acc: 0.5000, Val_Loss: 1.3511, Val_Acc: 0.7720
Epoch 13, Loss: 1.2589, Acc: 0.5000, Val_Loss: 1.2871, Val_Acc: 0.7840
Epoch 14, Loss: 1.2286, Acc: 0.4714, Val_Loss: 1.2314, Val_Acc: 0.7900
Epoch 15, Loss:

In [82]:
if __name__ == "__main__":
    # Assume all_datasets dictionary is already populated as in the previous steps
    dataset_name = 'Citeseer'  # Change this to Citeseer or Pubmed as needed
    data = load_processed_data_from_dict(all_datasets[dataset_name])

    # Model configuration based on the paper's setup
    num_features = data.x.size(1)
    num_classes = int(data.y.max()) + 1
    hidden_dim = 16  # Set the hidden dimension to 16 as per the paper
    num_views = 3

    # Initialize the model with the specified architecture and dimensions
    model = MAGCN(num_features, num_classes, num_views, hidden_dim)
    
    # Use Adam optimizer with the specified learning rate and weight decay
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # Train and test the model
    train_model(model, data, optimizer, epochs=2000, early_stopping_patience=100)
    test_model(model, data)

Epoch 1, Loss: 1.7988, Train Accuracy: 0.1417, Validation Loss: 1.7637, Validation Accuracy: 0.3060
Epoch 2, Loss: 1.7441, Train Accuracy: 0.2417, Validation Loss: 1.7156, Validation Accuracy: 0.3880
Epoch 3, Loss: 1.6648, Train Accuracy: 0.3750, Validation Loss: 1.6520, Validation Accuracy: 0.4880
Epoch 4, Loss: 1.5810, Train Accuracy: 0.4500, Validation Loss: 1.5770, Validation Accuracy: 0.5720
Epoch 5, Loss: 1.4979, Train Accuracy: 0.4167, Validation Loss: 1.5054, Validation Accuracy: 0.6260
Epoch 6, Loss: 1.4429, Train Accuracy: 0.4167, Validation Loss: 1.4391, Validation Accuracy: 0.6740
Epoch 7, Loss: 1.3084, Train Accuracy: 0.4667, Validation Loss: 1.3790, Validation Accuracy: 0.7100
Epoch 8, Loss: 1.2223, Train Accuracy: 0.5333, Validation Loss: 1.3226, Validation Accuracy: 0.7120
Epoch 9, Loss: 1.1668, Train Accuracy: 0.5333, Validation Loss: 1.2673, Validation Accuracy: 0.7160
Epoch 10, Loss: 1.1263, Train Accuracy: 0.5667, Validation Loss: 1.2153, Validation Accuracy: 0.7220

In [84]:
if __name__ == "__main__":
    # Assume all_datasets dictionary is already populated as in the previous steps
    dataset_name = 'Pubmed'  # Change this to Citeseer or Pubmed as needed
    data = load_processed_data_from_dict(all_datasets[dataset_name])

    # Model configuration based on the paper's setup
    num_features = data.x.size(1)
    num_classes = int(data.y.max()) + 1
    hidden_dim = 16  # Set the hidden dimension to 16 as per the paper
    num_views = 3

    # Initialize the model with the specified architecture and dimensions
    model = MAGCN(num_features, num_classes, num_views, hidden_dim)
    
    # Use Adam optimizer with the specified learning rate and weight decay
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # Train and test the model
    train_model(model, data, optimizer, epochs=2000, early_stopping_patience=100)
    test_model(model, data)

Epoch 1, Loss: 1.0977, Train Accuracy: 0.3333, Validation Loss: 1.0944, Validation Accuracy: 0.5120
Epoch 2, Loss: 1.0945, Train Accuracy: 0.4667, Validation Loss: 1.0900, Validation Accuracy: 0.5640
Epoch 3, Loss: 1.0893, Train Accuracy: 0.3667, Validation Loss: 1.0851, Validation Accuracy: 0.6580
Epoch 4, Loss: 1.0769, Train Accuracy: 0.5000, Validation Loss: 1.0780, Validation Accuracy: 0.6900
Epoch 5, Loss: 1.0683, Train Accuracy: 0.4333, Validation Loss: 1.0713, Validation Accuracy: 0.6760
Epoch 6, Loss: 1.0612, Train Accuracy: 0.5000, Validation Loss: 1.0634, Validation Accuracy: 0.6520
Epoch 7, Loss: 1.0559, Train Accuracy: 0.5500, Validation Loss: 1.0563, Validation Accuracy: 0.6640
Epoch 8, Loss: 1.0441, Train Accuracy: 0.4667, Validation Loss: 1.0502, Validation Accuracy: 0.6740
Epoch 9, Loss: 1.0269, Train Accuracy: 0.6167, Validation Loss: 1.0450, Validation Accuracy: 0.6780
Epoch 10, Loss: 1.0200, Train Accuracy: 0.6500, Validation Loss: 1.0404, Validation Accuracy: 0.6580